<a href="https://colab.research.google.com/github/dioskit/ML-test/blob/main/titanic_using_random_forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
test_path = "/content/drive/MyDrive/Colab Notebooks/titanic/test.csv"
train_path = "/content/drive/MyDrive/Colab Notebooks/titanic/train.csv"

Mounted at /content/drive


In [171]:
import pandas as  pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


In [172]:
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)
print(train_df.head(3))
test_df.head(3)

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q


In [173]:
def remove_sparse_cols(df):
    for col in df.columns:
        if df[col].isna().sum()/len(df) > 0.60:
            df.drop(col, axis=1, inplace=True)

# ? to ask a pro to what should i do when i have the embarked ?

- here im confused as i have no idea, so i made an assumpotion as max people embarked from __'S'__ so i assigned the nan values to __'S'__


# mapping

In [174]:
train_df.Sex = train_df.Sex.map({'male':1,'female':0})
train_df.Embarked = train_df.Embarked.map({'S':0,'C':-1,'Q':1})

In [175]:
round(train_df.Embarked.value_counts().idxmax())
# print(train_df.Embarked.isna().sum())

0

In [176]:
train_df.Age.fillna(train_df.Age.mean(),inplace=True)
train_df.Embarked.fillna(train_df.Embarked.mean(),inplace=True)

In [177]:
remove_sparse_cols(train_df)
train_df.isna().sum()

,0
PassengerId,0
Survived,0
Pclass,0
Name,0
Sex,0
Age,0
SibSp,0
Parch,0
Ticket,0
Fare,0


In [178]:
y = train_df.Survived
features = ['PassengerId', 'Pclass',  'Sex', 'Age', 'SibSp','Parch', 'Fare', 'Embarked']
X = train_df[features]
print(y.head())
X.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64


,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,3,1,22.0,1,0,7.2500,0.0
1,2,1,0,38.0,1,0,71.2833,-1.0
2,3,3,0,26.0,0,0,7.9250,0.0
3,4,1,0,35.0,1,0,53.1000,0.0
4,5,3,1,35.0,0,0,8.0500,0.0


In [179]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

In [180]:
train_X, val_X, train_y, val_y = train_test_split(X,y,random_state = 0)

In [181]:
train_X.isna().sum()

,0
PassengerId,0
Pclass,0
Sex,0
Age,0
SibSp,0
Parch,0
Fare,0
Embarked,0


In [182]:
rf_model = RandomForestRegressor(random_state=1)
rf_model.fit(train_X,train_y)
print(mean_absolute_error(val_y,rf_model.predict(val_X)))

0.25170403587443946


In [183]:
rf_model.fit(X,y)


RandomForestRegressor(random_state=1)

# now creating the result and the result csv


In [184]:
#mapping
test_df.Sex = test_df.Sex.map({'male':1,'female':0})
test_df.Embarked = test_df.Embarked.map({'S':0,'C':-1,'Q':1})

#removing and filling nan values
test_df.Age.fillna(test_df.Age.mean(),inplace=True)
test_df.Embarked.fillna(test_df.Embarked.mean(),inplace=True)
test_df.Fare.fillna(test_df.Fare.mean(),inplace=True)


In [188]:

remove_sparse_cols(test_df)

#features
features = ['PassengerId', 'Pclass',  'Sex', 'Age', 'SibSp','Parch', 'Fare', 'Embarked']
X = test_df[features]

In [151]:
X.isna().sum()

,0
PassengerId,0
Pclass,0
Sex,0
Age,0
SibSp,0
Parch,0
Fare,1
Embarked,0


In [189]:
result = rf_model.predict(X)

test_df['Survived'] = np.round(result).astype(int)


final_data = test_df[['PassengerId','Survived']]
final_data.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


# to  csv

In [192]:
final_data.to_csv("SUBMISSION.csv", index=False, encoding='utf-8')